# Intelligent Query Processing in SQL Server 2019 - Table Variable Deferred Compilation

## Step 1 - Create the stored procedure
This procedure uses a table variable populated from a user table and then joins it with a user table to provide output. T-SQL functions like COUNT and SUM are often seen in analytic queries that benefit from Intelligent Query Processing. Note: In this example the TOP 1 T-SQL syntax is used so that the procedure only produces 1 row. This is only done to make the output easier to read using this workshop and demo since this procedure will be executed multiple times. Normal execution of this procedure may not include TOP.

There is also a query to bring all pages from Sales.Invoices into buffer pool cache to ensure the comparisons on Step 2 and 3 are fair.


In [10]:
USE WideWorldImporters
GO
CREATE or ALTER PROCEDURE [Sales].[CustomerProfits]
AS
BEGIN
-- Declare the table variable
DECLARE @ilines TABLE
(	[InvoiceLineID] [int] NOT NULL primary key,
	[InvoiceID] [int] NOT NULL,
	[StockItemID] [int] NOT NULL,
	[Description] [nvarchar](100) NOT NULL,
	[PackageTypeID] [int] NOT NULL,
	[Quantity] [int] NOT NULL,
	[UnitPrice] [decimal](18, 2) NULL,
	[TaxRate] [decimal](18, 3) NOT NULL,
	[TaxAmount] [decimal](18, 2) NOT NULL,
	[LineProfit] [decimal](18, 2) NOT NULL,
	[ExtendedPrice] [decimal](18, 2) NOT NULL,
	[LastEditedBy] [int] NOT NULL,
	[LastEditedWhen] [datetime2](7) NOT NULL
)

-- Insert all the rows from InvoiceLines into the table variable
INSERT INTO @ilines SELECT * FROM Sales.InvoiceLines

-- Find my total profile by customer
SELECT TOP 1 COUNT(i.CustomerID) as customer_count, SUM(il.LineProfit) as total_profit
FROM Sales.Invoices i
INNER JOIN @ilines il
ON i.InvoiceID = il.InvoiceID
GROUP By i.CustomerID
END
GO
-- Pull these pages into cache to make the comparison fair based on a warm buffer pool cache
SELECT COUNT(*) FROM Sales.Invoices

Commands completed successfully.

Commands completed successfully.

(1 row affected)

Total execution time: 00:00:00.013

(No column name)
70510


## Step 2 - Run the stored procedure with database compatibility of 130
You have been told this procedure executes fairly quickly with a single execution in a few seconds but over several iterations the total duration, over 20 seconds, is not acceptable to the application.

The script will ensure the database is in a compatibility mode that is less than 150 so Intelligent Query Processing will NOT be enabled. The script also turns off rowcount messages to be returned to the client to reduce network traffic for this test. Then the script executes the stored procedure. Notice the syntax of **GO 25**. This is a client tool tip that says to run the batch 25 times (avoids having to construct a loop).

When you click Play to run the script look for these messages on the total elapsted time (your time may vary)

<pre>Beginning execution loop
Batch execution completed 25 times...
Total execution time: 00:00:40.3520665</pre>


In [11]:
USE master
GO
ALTER DATABASE wideworldimporters SET compatibility_level = 130
GO
USE WideWorldImporters
GO
SET NOCOUNT ON
GO
EXEC [Sales].[CustomerProfits]
GO 25
SET NOCOUNT OFF
GO

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Beginning execution loop

...

customer_count,total_profit
406,147225.15


customer_count,total_profit
406,147225.15


customer_count,total_profit
406,147225.15


customer_count,total_profit
406,147225.15


customer_count,total_profit
406,147225.15


## Step 3 - Run the stored procedure with database compatibility of 150
Now let's run the same exact test but with database compatibility of 150. You will not make any changes to the stored procedure.

Notice this is the same script except database compatibility of 150 is used. This time, the query processor in SQL Server will enable table variable deferred compilation so a better query plan can be chosen

 The script should execute far faster than before. Your speeds can vary but should be 15 seconds or less.

 When you click Play to run the script look for these messages on the total elapsted time (your time may vary)

<pre>Beginning execution loop
Batch execution completed 25 times...
Total execution time: 00:00:10.9975239</pre>


In [12]:
USE master
GO
ALTER DATABASE wideworldimporters SET compatibility_level = 150
GO
USE WideWorldImporters
GO
SET NOCOUNT ON
GO
EXEC [Sales].[CustomerProfits]
GO 25
SET NOCOUNT OFF
GO

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Beginning execution loop

...

customer_count,total_profit
406,147225.15


customer_count,total_profit
406,147225.15


customer_count,total_profit
406,147225.15


customer_count,total_profit
406,147225.15


customer_count,total_profit
406,147225.15


## Step 4: Restore dbcompat of WideWorldImporters
Restore the dbcompat back to 130 for WideWorldImporters in case other examples rely on it

In [4]:
USE master
GO
ALTER DATABASE wideworldimporters SET compatibility_level = 130
GO

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.006